In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200413081440-0000
KERNEL_ID = 1ea1c486-7e5c-44cb-8689-bcd4d8204196
Libraries imported.


In [ ]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [39]:
df = pd.read_html(r'https://www.mapsofindia.com/pincode/india/uttar-pradesh/kanpur-nagar/', header = 1)
df = pd.DataFrame(df[0])
df.head()

,Location,Pincode,State,District
0,Agriculture College,208002,Uttar Pradesh,Kanpur Nagar
1,Amour,209401,Uttar Pradesh,Kanpur Nagar
2,Anand Nagar,208011,Uttar Pradesh,Kanpur Nagar
3,Anwarganj,208003,Uttar Pradesh,Kanpur Nagar
4,Aranjhami,209401,Uttar Pradesh,Kanpur Nagar


In [40]:
df.shape

(175, 4)

latitude of Barra, Kanpur, Uttar Pradesh, India is 26.427971, and the longitude is 80.296242.

Arya Nagar Pincode is 208002 With 26.51699 Latitude and 80.31155 Longitude

JNVM Pincode is 208024 With 26.4933 Latitude and 80.2847 Longitude

In [2]:

CLIENT_ID = 'FZV1F5DFXYG3HQX51UIYYUGFZUZASDZ00OM002X5UFT5U0VH' # your Foursquare ID
CLIENT_SECRET = '554WLBIL5FFK3IDA13QH15OMCERX5KZRBZG1M5FKRXH2X5E4' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FZV1F5DFXYG3HQX51UIYYUGFZUZASDZ00OM002X5UFT5U0VH
CLIENT_SECRET:554WLBIL5FFK3IDA13QH15OMCERX5KZRBZG1M5FKRXH2X5E4


In [23]:
radius=5000
LIMIT =100
lat= 26.4933
lng= 80.2847  
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5e942a46882fc76a3346ab8a'},
 'response': {'headerLocation': 'Kānpur',
  'headerFullLocation': 'Kānpur',
  'headerLocationGranularity': 'city',
  'totalResults': 13,
  'suggestedBounds': {'ne': {'lat': 26.538300045000046,
    'lng': 80.33488625770737},
   'sw': {'lat': 26.448299954999957, 'lng': 80.23451374229263}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d9cb59ec540236a4e187f4b',
       'name': 'Upper Crust',
       'location': {'address': '112/368 F',
        'crossStreet': 'Swarup Nagar',
        'lat': 26.48222458226252,
        'lng': 80.32085672413587,
        'labeledLatLngs': [{'label': 'display',
          'lat': 26.48222458226252,
          'lng': 80.32085672413587}],
        'distance': 3807,
        'postalCode': '

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = result['response']['groups'][0]['items']
    
venues_df = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance']
venues_df = venues_df.loc[:, filtered_columns]

# filter the category for each row
venues_df['venue.categories'] = venues_df.apply(get_category_type, axis=1)

venues_df.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance
0,Upper Crust,Bakery,26.482225,80.320857,3807
1,Hotel Royal Cliff,Hotel,26.479289,80.314855,3385
2,"Aromas, Arya Nagar",Fast Food Restaurant,26.482953,80.318024,3514
3,Rave @ Moti,Shopping Mall,26.481579,80.300102,2014
4,Rave 3,Shopping Mall,26.492315,80.327160,4231


In [34]:
venues_df = venues_df.rename(columns = {'venue.name':'Venue','venue.categories':'Category','venue.location.lat':'Latitude','venue.location.lng':'Longitude', 'venue.location.distance': 'Distance'})
venues_df.head()

,Venue,Category,Latitude,Longitude,Distance
0,Upper Crust,Bakery,26.482225,80.320857,3807
1,Hotel Royal Cliff,Hotel,26.479289,80.314855,3385
2,"Aromas, Arya Nagar",Fast Food Restaurant,26.482953,80.318024,3514
3,Rave @ Moti,Shopping Mall,26.481579,80.300102,2014
4,Rave 3,Shopping Mall,26.492315,80.327160,4231


In [2]:
#Reading in the table and setting the header to the first row
df = pd.read_html(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)
df = pd.DataFrame(df[0])
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
cdf= df[df['Borough']!='Not assigned']
cdf.reset_index(inplace = True, drop = True)

In [4]:
cdf_fil= cdf[cdf['Borough'].str.contains('Toronto')]
cdf_fil.head(20)

,Postal code,Borough,Neighborhood
2,M5A,Downtown Toronto,Regent Park / Harbourfront
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
9,M5B,Downtown Toronto,"Garden District, Ryerson"
15,M5C,Downtown Toronto,St. James Town
19,M4E,East Toronto,The Beaches
20,M5E,Downtown Toronto,Berczy Park
24,M5G,Downtown Toronto,Central Bay Street
25,M6G,Downtown Toronto,Christie
30,M5H,Downtown Toronto,Richmond / Adelaide / King
31,M6H,West Toronto,Dufferin / Dovercourt Village


In [5]:
#Replacing the "/" with a ","
cdf_fil['Neighborhood'] = cdf_fil['Neighborhood'].replace("/", ",", regex = True)
cdf_fil.head()

/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Postal code,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
9,M5B,Downtown Toronto,"Garden District, Ryerson"
15,M5C,Downtown Toronto,St. James Town
19,M4E,East Toronto,The Beaches


In [6]:
for i in cdf_fil['Neighborhood']:
    if i=='NaN':
      print (i)

In [7]:
cdf_fil.shape

(39, 3)

In [8]:
## Importing the CSV file with geo-loc data
geo = pd.read_csv(r'http://cocl.us/Geospatial_data', header = 0)
geo.shape

(103, 3)

In [9]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
pdf= pd.DataFrame(columns= ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])

In [11]:
for i in cdf['Postal code']:
    for j in geo['Postal Code']:
        if i==j:
               pdf['Postal Code']=cdf['Postal code']
               pdf['Borough']=cdf['Borough']
               pdf['Neighborhood']=cdf['Neighborhood']
               pdf['Latitude']= geo['Latitude']
               pdf['Longitude']=geo['Longitude']

In [12]:
pdf= pdf[pdf['Borough'].str.contains('Toronto')]
pdf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.763573,-79.188711
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.773136,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
15,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
19,M4E,East Toronto,The Beaches,43.786947,-79.385975


In [13]:
len(pdf['Neighborhood'].unique())

39

In [14]:
pdf['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [15]:
pdf.shape

(39, 5)

In [16]:
CLIENT_ID = 'FZV1F5DFXYG3HQX51UIYYUGFZUZASDZ00OM002X5UFT5U0VH' # your Foursquare ID
CLIENT_SECRET = '554WLBIL5FFK3IDA13QH15OMCERX5KZRBZG1M5FKRXH2X5E4' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FZV1F5DFXYG3HQX51UIYYUGFZUZASDZ00OM002X5UFT5U0VH
CLIENT_SECRET:554WLBIL5FFK3IDA13QH15OMCERX5KZRBZG1M5FKRXH2X5E4


In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:

venues_list=[]
radius=500
LIMIT =100

for name, lat, lng in zip(pdf['Neighborhood'], pdf['Latitude'], pdf['Longitude']):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            x['venue']['name'], 
            x['venue']['location']['lat'], 
            x['venue']['location']['lng'],  
            x['venue']['categories'][0]['name']) for x in results])

ven_pdf = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
ven_pdf.columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'venue', 
                  'Venue latitude', 
                  'Venue longitude', 
                  'categories']

ven_pdf.head(20)

,Neighborhood,Latitude,Longitude,venue,Venue latitude,Venue longitude,categories
0,Regent Park / Harbourfront,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
1,Regent Park / Harbourfront,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
2,Regent Park / Harbourfront,43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
3,Regent Park / Harbourfront,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,Regent Park / Harbourfront,43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
5,Regent Park / Harbourfront,43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
6,Regent Park / Harbourfront,43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
7,Queen's Park / Ontario Provincial Government,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
8,Queen's Park / Ontario Provincial Government,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant
9,Queen's Park / Ontario Provincial Government,43.773136,-79.239476,Thai One On,43.774468,-79.241268,Thai Restaurant


In [19]:
ven_pdf.groupby('Neighborhood').count()

,Latitude,Longitude,venue,Venue latitude,Venue longitude,categories
Neighborhood,,,,,,
Berczy Park,1,1,1,1,1,1
Brockton / Parkdale Village / Exhibition Place,40,40,40,40,40,40
Business reply mail Processing CentrE,4,4,4,4,4,4
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,18,18,18,18,18,18
Central Bay Street,8,8,8,8,8,8
Christie,2,2,2,2,2,2
Church and Wellesley,7,7,7,7,7,7
Commerce Court / Victoria Hotel,1,1,1,1,1,1
Davisville,4,4,4,4,4,4


In [20]:
# one hot encoding
onehot = pd.get_dummies(ven_pdf[['categories']], prefix="", prefix_sep="")

In [21]:
# add neighborhood column back to dataframe
onehot['Neighborhood'] = ven_pdf['Neighborhood']

In [22]:
onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Social Club,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
# move neighborhood column to the first column
onehot.set_index('Neighborhood', inplace = True)

In [24]:
onehot.head(20)

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Social Club,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [25]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Social Club,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0

In [26]:
grouped.drop('Neighborhood', axis=1, inplace=True)
grouped.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Social Club,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.000,1.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.025,0.025,0.0,0.0,0.0,0.0,0.0,0.

In [27]:
# set number of clusters
clusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 2, 0, 1, 0, 0], dtype=int32)

In [28]:
grouped.shape
ven_pdf.shape
pdf.shape
len(kmeans.labels_)

39

In [29]:
groupc = onehot.groupby('Neighborhood').mean().reset_index()
groupc.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Social Club,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.000,1.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,

In [30]:
num_top_venues = 5

for c in groupc['Neighborhood']:
    print("----"+c+"----")
    temp = groupc[groupc['Neighborhood'] == c].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
               venue  freq
0          Cafeteria   1.0
1            Airport   0.0
2  Mobile Phone Shop   0.0
3     Lingerie Store   0.0
4       Liquor Store   0.0


----Brockton / Parkdale Village / Exhibition Place----
         venue  freq
0         Café  0.10
1  Coffee Shop  0.08
2    Gastropub  0.05
3      Brewery  0.05
4       Bakery  0.05


----Business reply mail Processing CentrE----
                venue  freq
0   Mobile Phone Shop  0.25
1         Pizza Place  0.25
2      Sandwich Place  0.25
3            Bus Line  0.25
4  Miscellaneous Shop  0.00


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2                 Spa  0.06
3       Garden Center  0.06
4              Garden  0.06


----Central Bay Street----
           venue  freq
0           Bank  0.12
1  Grocery Store  0.12
2       Pharmacy  0.12
3    C

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

# create columns according to number of top venues
columns = ['Neighborhood']

indicators = ['st', 'nd', 'rd']
for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))


In [33]:
# create a new dataframe
sorted = pd.DataFrame(columns=columns)
sorted['Neighborhood'] = groupc['Neighborhood']

for ind in np.arange(groupc.shape[0]):
    sorted.iloc[ind, 1:] = return_most_common_venues(groupc.iloc[ind, :], num_top_venues)

sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cafeteria,Yoga Studio,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Ice Cream Shop,Bookstore,Italian Restaurant
2,Business reply mail Processing CentrE,Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Yoga Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
3,CN Tower / King and Spadina / Railway Lands / ...,Light Rail Station,Yoga Studio,Auto Workshop,Recording Studio,Brewery,Skate Park,Smoke Shop,Spa,Farmers Market,Fast Food Restaurant
4,Central Bay Street,Home Service,Butcher,Coffee Shop,Grocery Store,Discount Store,Pharmacy,Pizza Place,Bank,Eastern European Restaurant,Dumpling Restaurant


In [34]:
# add clustering labels
sorted.insert(0, 'Cluster Labels', kmeans.labels_)
sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Berczy Park,Cafeteria,Yoga Studio,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
1,0,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Ice Cream Shop,Bookstore,Italian Restaurant
2,0,Business reply mail Processing CentrE,Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Yoga Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
3,0,CN Tower / King and Spadina / Railway Lands / ...,Light Rail Station,Yoga Studio,Auto Workshop,Recording Studio,Brewery,Skate Park,Smoke Shop,Spa,Farmers Market,Fast Food Restaurant
4,0,Central Bay Street,Home Service,Butcher,Coffee Shop,Grocery Store,Discount Store,Pharmacy,Pizza Place,Bank,Eastern European Restaurant,Dumpling Restaurant


In [35]:
sorted.set_index('Neighborhood')

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,,
Berczy Park,3,Cafeteria,Yoga Studio,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
Brockton / Parkdale Village / Exhibition Place,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Ice Cream Shop,Bookstore,Italian Restaurant
Business reply mail Processing CentrE,0,Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Yoga Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,0,Light Rail Station,Yoga Studio,Auto Workshop,Recording Studio,Brewery,Skate Park,Smoke Shop,Spa,Farmers Market,Fast Food Restaurant
Central Bay Street,0,Home Service,Butcher,Coffee Shop,Grocery Store,Discount Store,Pharmacy,Pizza Place,Bank,Eastern European Restaurant,Dumpling Restaurant
Christie,2,Park,Food & Drink Shop,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
Church and Wellesley,0,Pizza Place,Coffee Shop,Chinese Restaurant,Sandwich Place,Discount Store,Middle Eastern Restaurant,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
Commerce Court / Victoria Hotel,1,Restaurant,Yoga Studio,College Rec Center,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
Davisville,0,Park,Construction & Landscaping,Basketball Court,Bakery,Yoga Studio,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [36]:
merged = pdf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.763573,-79.188711,0,Electronics Store,Medical Center,Breakfast Spot,Intersection,Rental Car Location,Bank,Mexican Restaurant,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.773136,-79.239476,0,Lounge,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Bank,Bakery,Gas Station,Dog Run
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0,Skating Rink,Café,College Stadium,General Entertainment,Gay Bar,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
15,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,0,Fast Food Restaurant,Chinese Restaurant,Nail Salon,Pharmacy,Breakfast Spot,Bank,Supermarket,Camera Store,Pizza Place,Sandwich Place
19,M4E,East Toronto,The Beaches,43.786947,-79.385975,0,Japanese Restaurant,Chinese Restaurant,Bank,Café,Yoga Studio,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [38]:
# create map
for lat, lon in zip(merged['Latitude'], merged['Longitude']):
    map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
       
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters